In [2]:
import tensorflow as tf
import sys
import pickle
import numpy as np
import pandas as pd

#sys.path.append('/content/drive/MyDrive/Project_2')

##########################################################################################

# loading variables required
embedding_matrix = pickle.load(open('C:/Users/rpris/OneDrive/Desktop/Project_2/embedding_matrix.pkl','rb'))
embedding_matrix_per = pickle.load(open('C:/Users/rpris/OneDrive/Desktop/Project_2/embedding_matrix_per.pkl','rb'))

token_perturbation = pickle.load(open("C:/Users/rpris/OneDrive/Desktop/Project_2/token_perturbation-reg.pkl",'rb'))
token_correct = pickle.load(open("C:/Users/rpris/OneDrive/Desktop/Project_2/token_correct-reg.pkl", 'rb'))

v_perturbation = len(token_perturbation.word_index.keys()) +1
v_correct=len(token_correct.word_index.keys())  + 1    

##########################################################################################

# importing our custom attention_model library

import monotonic_attention

##########################################################################################

#defining model
e_vocab_size = v_perturbation
d_vocab_size = v_correct
embedding_dim_e = 300
embedding_dim_d = 300
i_length = 16
o_length = 16
enc_units = 300
dec_units = 300
score_fun = 'parallel'
att_units = 300

attention = monotonic_attention.attention_model(e_vocab_size, d_vocab_size, embedding_dim_e,embedding_dim_d, 
        i_length, o_length, enc_units, dec_units ,score_fun ,att_units, embedding_matrix, embedding_matrix_per)


attention.build((None,512,16))    

attention.load_weights("C:/Users/rpris/OneDrive/Desktop/Project_2/wts/attention/weights-21-0.2798.hdf5")

##########################################################################################


In [25]:
encoder = attention.layers[0]
decoder_layer = attention.layers[1]
onestep = decoder_layer.layers[0]

def beam(enc_inp):
    #inference

    e_input=[]
    for i in enc_inp.split():
        if token_perturbation.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(token_perturbation.word_index.get(i))

    e_input = tf.keras.preprocessing.sequence.pad_sequences([e_input], maxlen=16, padding='post')


    e_output, e_hidden, e_cell = encoder(e_input,0)

    d_hidden = e_hidden
    d_cell = e_cell                   #final encoder state is equal to initial decoder state

    #initial decoder input is start
    d_input = tf.expand_dims([token_correct.word_index['<start>']],0)
    attention_weights = np.zeros((1,16), dtype='float32')
    attention_weights[:,0] = 1

    predicted, d_hidden, d_cell, attention_weights, context_vector = onestep(d_input, e_output, d_hidden, d_cell, attention_weights)
    topk = tf.math.top_k(predicted[0], k=3)
    top3_index = topk.indices.numpy()
    top3_prob = topk.values.numpy()

    prev_index = []
    for i in top3_index:
        prev_index.append([int(i)])


    for word in range(1,16):
        product = []
        for i in range(3):
            d_input = tf.expand_dims([top3_index[i]],0)
            
            predicted, dh, dc, aw, context_vector = onestep(d_input, e_output, d_hidden,                                                                                     d_cell, attention_weights)
            predicted = np.array(predicted[0])
        
            product.append(predicted + top3_prob[i])
    
        #restoring hidden states and weights (of the previous input ) for the next iteration
        d_input = tf.expand_dims([top3_index[0]],0)   
        predicted, d_hidden, d_cell, attention_weights, context_vector = onestep(d_input, e_output, d_hidden,                                                                                     d_cell, attention_weights)
        # getting the conditional probability
        product = np.stack(product, axis=0).reshape(-1,)
        
        #getting top 3
        topk = tf.math.top_k(product, k=3)
        top3_index_temp = topk.indices.numpy()
        top3_prob  = topk.values.numpy()
    
        top3_index = []    
        temp = []
        
        #getting the absolute index since we stacked all the three outputs together above
        for i in top3_index_temp:

            if i>28466 and i<=56933: 
                top3_index.append(i-28467)           #2nd group
                tmp = prev_index[1].copy()         # tmp = prev_index[1]--> this was changing the original list 
                tmp.append(i-28467)
                
            elif i>56933:
                top3_index.append(i - 56934)         #3rd group
                tmp = prev_index[2].copy()              
                tmp.append(i-56934)
                
            else: 
                top3_index.append(i)                 #1st group
                tmp = prev_index[0].copy()
                tmp.append(i)

            temp.append(tmp)
    
        prev_index = temp
    
    try:
        xx = prev_index[0][:prev_index[0].index(18896)]
    except:
        xx = prev_index[0]

    
    translation = ""
    for i in xx:
        translation = translation + token_correct.index_word[i] + ' '
    
    return translation

In [36]:
def inference_att(enc_inp):

    translation=""
    
    e_input=[]
    for i in enc_inp.split():
        if token_perturbation.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(token_perturbation.word_index.get(i))
    #e_input = [token_perturbation.word_index[i] for i in enc_inp.split(" ")]
    e_input = tf.keras.preprocessing.sequence.pad_sequences([e_input], maxlen=16, padding='post')
    
    
    e_output, e_hidden, e_cell = encoder(e_input,0)
    
    d_hidden = e_hidden
    d_cell = e_cell                   #final encoder state is equal to initial decoder state
    
    #initial decoder input is start
    d_input = tf.expand_dims([token_correct.word_index['<start>']],0)

    attention_weights = np.zeros((1,16), dtype='float32')
    attention_weights[:,0] = 1

    for word in range(16):
        
        predicted, d_hidden, d_cell, attention_weights, context_vector = onestep(d_input, e_output, d_hidden, d_cell,
                                                                                 attention_weights)        
        
        #attention_weights = tf.reshape(attention_weights, (-1,))  #creating a 1d tensor attention weights to store them
        #attention_plot[word] = attention_weights.numpy()
        
        #making predictions
        predicted_word_index = tf.argmax(predicted[0]).numpy()  #it will give the highest probability word
      

        #stopping when reaching "<end>"
        if token_correct.index_word[predicted_word_index] == '<end>':
            return translation #, attention_plot
        
        #appending the result with predicted words
        translation = translation + token_correct.index_word[predicted_word_index] + ' '
        
        # the predicted ID is fed back into the model
        d_input = tf.expand_dims([predicted_word_index], 0)
        
    return translation #, attention_plot

In [37]:
inference_att('i have pen')

'i have a pen '

In [18]:
import math

In [49]:
def beamsearch(enc_inp, beam_width):
    
    start = [token_correct.word_index['<start>']]

    sequences = [[start, 0]]
    e_input=[]
    for i in enc_inp.split():
        if token_perturbation.word_index.get(i) == None:
            e_input.append(0)
        else:
            e_input.append(token_perturbation.word_index.get(i))

    e_input = tf.keras.preprocessing.sequence.pad_sequences([e_input], maxlen=16, padding='post')


    e_output, e_hidden, e_cell = encoder(e_input,0)

    d_hidden = e_hidden
    d_cell = e_cell                   #final encoder state is equal to initial decoder state

    #initial decoder input is start
    d_input = tf.expand_dims([token_correct.word_index['<start>']],0)
    attention_weights = np.zeros((1,16), dtype='float32')
    attention_weights[:,0] = 1
    
    #img_features = Xnet_Features[image]
    #img_features = encoder_model.predict(img_features)
    finished_seq = []
    
    for i in range(20):
        all_candidates = []
        new_seq = []
        for s in sequences:

            #text_input = pad_sequences([s[0]], 155, padding='post')
            predictions, d_hidden, d_cell, attention_weights, context_vector = onestep(d_input, e_output,                                                                         d_hidden, d_cell, attention_weights)
            top_words = np.argsort(predictions[0])[-beam_width:]
            seq, score = s
            
            for t in top_words:
                candidates = [seq + [t], score - math.log(predictions[0][t])]
                all_candidates.append(candidates)
                
        sequences = sorted(all_candidates, key = lambda l: l[1])[:beam_width]
        # checks for 'endseq' in each seq in the beam
        count = 0
        for seq,score in sequences:
            if seq[len(seq)-1] == token_correct.word_index['<end>']:
                score = score/len(seq)   # normalized
                finished_seq.append([seq, score])
                count+=1
            else:
                new_seq.append([seq, score])
        beam_width -= count
        sequences = new_seq
        
        # if all the sequences reaches its end before 155 timesteps
        if not sequences:
            break
        else:
            continue
        
    sequences = finished_seq[-1] 
    rep = sequences[0]
    score = sequences[1]
    temp = []
    rep.pop(0)
    for word in rep:
        if word != token_correct.word_index['<end>']:
            temp.append(token_correct.index_word[word])
        else:
            break    
    rep = ' '.join(e for e in temp)        
    
    return rep, score

In [50]:
beamsearch('we are going to hospital', 3)

('we we to the', -2.3801649549686075)

## Train

In [24]:
train = pickle.load(open('C:/Users/rpris/OneDrive/Desktop/Project_2/train-reg.pkl', 'rb'))

In [28]:
from tqdm import tqdm

In [30]:
sample_train = train.sample(1000)
result = []
for enc_inp,dec_inp,_ in tqdm(sample_train.values):
    pred = beam(enc_inp)
    result.append(pred)
sample_train['correct_output'] = sample_train['decoder_output']
sample_train['beam_predicted_output'] = result
sample_train = sample_train.drop(['decoder_input', 'decoder_output'], axis=1)
sample_train.head(10)

100%|██████████| 1000/1000 [10:19<00:00,  1.61it/s]


,encoder_input,correct_output,beam_predicted_output
80389,a key that is it that is what he talking about,a key that is it that is what he was talking a...,a key is that is that is what he is talking ab...
140069,than what do you want to stay with me for,then what do you want to stay with me for <end>,then what do you want to stay with me
100549,in view of all circumstances commissioner has ...,in view of all the circumstances the commissio...,in view of all the circumstances council has d...
186633,what the status of the decryption effort,what is the status of the decryption effort <end>,what is the status of the decryption effort
98686,look at me i can not even fly out of an window,look at me i can not even fly out of a window ...,look at me i can not even fly out of a window
192556,i thought they was carrying something,i thought he was carrying something <end>,i thought he was carrying something
27749,eric wanted to see the island and the dinosaur...,eric wanted to see the island and the dinosaur...,see the island and the dinosaurs so ben found ...
130066,you should not believe what i say when i am wi...,you should not believe what i say when i am wi...,you should not believe what i say when i am wi...
165992,it me they wanted not you,it is me they wanted not you <end>,it is me they wanted not
127474,oh for chrissake she an actor,oh for chrissake she was an actor <end>,oh for chrissake she was an


In [31]:
import nltk.translate.bleu_score as bleu

In [33]:
train_bleu = []
for enc_inp,correct,pred in sample_train.values:
    
    correct = correct.split()[:-1]    #removing end
    pred = pred.split()
    
    if len(correct) == len(pred):
        train_bleu.append(bleu.sentence_bleu([correct],pred))
        
print("BLEU Score of train dataset with beam-search is",sum(train_bleu)/len(train_bleu))

BLEU Score of train dataset with beam-search is 0.9017522264103863


In [38]:
result = []
for enc_inp,dec_inp,_ in tqdm(sample_train.values):
    pred = inference_att(enc_inp)
    result.append(pred)

sample_train['greedy_predicted_output'] = result
sample_train.head(10)

100%|██████████| 1000/1000 [01:46<00:00,  9.42it/s]


,encoder_input,correct_output,beam_predicted_output,greedy_predicted_output
80389,a key that is it that is what he talking about,a key that is it that is what he was talking a...,a key is that is that is what he is talking ab...,a key that is it that is what he is talking ab...
140069,than what do you want to stay with me for,then what do you want to stay with me for <end>,then what do you want to stay with me,then what do you want to stay with me for
100549,in view of all circumstances commissioner has ...,in view of all the circumstances the commissio...,in view of all the circumstances council has d...,in view of all the circumstances the commissio...
186633,what the status of the decryption effort,what is the status of the decryption effort <end>,what is the status of the decryption effort,what is the status of the decryption effort
98686,look at me i can not even fly out of an window,look at me i can not even fly out of a window ...,look at me i can not even fly out of a window,look at me i can not even fly out of a window
192556,i thought they was carrying something,i thought he was carrying something <end>,i thought he was carrying something,i thought he was carrying something
27749,eric wanted to see the island and the dinosaur...,eric wanted to see the island and the dinosaur...,see the island and the dinosaurs so ben found ...,see the island and the dinosaurs so ben found ...
130066,you should not believe what i say when i am wi...,you should not believe what i say when i am wi...,you should not believe what i say when i am wi...,you should not believe what i say when i am wi...
165992,it me they wanted not you,it is me they wanted not you <end>,it is me they wanted not,it is me they wanted not you
127474,oh for chrissake she an actor,oh for chrissake she was an actor <end>,oh for chrissake she was an,oh for chrissake she was an


In [40]:
train_bleu_gr = []
for enc_inp,correct,pred,gr in sample_train.values:
    
    correct = correct.split()[:-1]    #removing end
    gr = gr.split()
    
    if len(correct) == len(gr):
        train_bleu_gr.append(bleu.sentence_bleu([correct],gr))
        
print("BLEU Score of train dataset with greedy-search is",sum(train_bleu_gr)/len(train_bleu_gr))

BLEU Score of train dataset with greedy-search is 0.9224930633097495


## Validation

In [41]:
validation = pickle.load(open('C:/Users/rpris/OneDrive/Desktop/Project_2/validation-reg.pkl', 'rb'))

In [42]:
sample_validation = validation.sample(1000)
result = []
for enc_inp,dec_inp,_ in tqdm(sample_validation.values):
    pred = beam(enc_inp)
    result.append(pred)
sample_validation['correct_output'] = sample_validation['decoder_output']
sample_validation['beam_predicted_output'] = result
sample_validation = sample_validation.drop(['decoder_input', 'decoder_output'], axis=1)
sample_validation.head(10)

100%|██████████| 1000/1000 [08:41<00:00,  1.92it/s]


,encoder_input,correct_output,beam_predicted_output
67940,but it not worth more than fifteen,but it is not worth more than fifteen <end>,but it is not worth more than fifteen
109011,lecter there is no correlation in literature b...,lecter there is no correlation in the literatu...,lecter there is no correlation in literature b...
162359,because it sentimental tacky crap that why not,because it is sentimental tacky crap that is w...,because it is frightening shootin that crap is...
201411,how come you all ai not sendin them to same sc...,how come you all ai not sendin them to the sam...,how come you all ai not thee them to the same
195977,barton i am afraid it not a good time,barton i am afraid it is not a good time <end>,barton i am afraid it is not a good time
132977,we waiting hubert,we are waiting hubert <end>,we are waiting
11931,do you know what friend you have in jesus son,do you know what a friend you have in jesus so...,do you know what friend you have in the warren...
190386,we not all hopped up on benzedrine orson i som...,we are not all hopped up on benzedrine orson i...,we not all hopped up filling on ganz i got eac...
197686,tasting and enjoying life the only thing of va...,tasting and enjoying life is the only thing of...,tasting and enjoying the life the thing of val...
205344,someone trying to kill eleanor,someone is trying to kill eleanor <end>,someone is trying to


In [44]:
validation_bleu = []
for enc_inp,correct,pred in sample_validation.values:
    
    correct = correct.split()[:-1]    #removing end
    pred = pred.split()

    if len(correct) == len(pred):
        validation_bleu.append(bleu.sentence_bleu([correct],pred))
        
print("BLEU Score of validation dataset with beam-search is",sum(validation_bleu)/len(validation_bleu))

BLEU Score of validation dataset with beam-search is 0.7387638566773894


In [46]:
result = []
for enc_inp,dec_inp,_ in tqdm(sample_validation.values):
    gr = inference_att(enc_inp)
    result.append(gr)

sample_validation['greedy_predicted_output'] = result

sample_validation.head(10)

100%|██████████| 1000/1000 [01:31<00:00, 10.97it/s]


,encoder_input,correct_output,beam_predicted_output,greedy_predicted_output
67940,but it not worth more than fifteen,but it is not worth more than fifteen <end>,but it is not worth more than fifteen,but it is not worth more than fifteen
109011,lecter there is no correlation in literature b...,lecter there is no correlation in the literatu...,lecter there is no correlation in literature b...,lecter there is no correlation in the literatu...
162359,because it sentimental tacky crap that why not,because it is sentimental tacky crap that is w...,because it is frightening shootin that crap is...,because it is frightening shootin that crap is...
201411,how come you all ai not sendin them to same sc...,how come you all ai not sendin them to the sam...,how come you all ai not thee them to the same,how come you all ai not thee them to the same ...
195977,barton i am afraid it not a good time,barton i am afraid it is not a good time <end>,barton i am afraid it is not a good time,barton i am afraid it is not a good time
132977,we waiting hubert,we are waiting hubert <end>,we are waiting,we were waiting
11931,do you know what friend you have in jesus son,do you know what a friend you have in jesus so...,do you know what friend you have in the warren...,do you know what friend you have in the jesus ...
190386,we not all hopped up on benzedrine orson i som...,we are not all hopped up on benzedrine orson i...,we not all hopped up filling on ganz i got eac...,all not all the up on throw ahold i some of us...
197686,tasting and enjoying life the only thing of va...,tasting and enjoying life is the only thing of...,tasting and enjoying the life the thing of val...,tasting and enjoying life the only thing of va...
205344,someone trying to kill eleanor,someone is trying to kill eleanor <end>,someone is trying to,someone is trying to kill


In [48]:
validation_bleu_gr = []
for enc_inp,correct,pred,gr in sample_validation.values:
    
    correct = correct.split()[:-1]    #removing end
    gr = gr.split()

    if len(correct) == len(gr):
        validation_bleu_gr.append(bleu.sentence_bleu([correct],gr))
        
print("BLEU Score of validation dataset with greedy-search is",sum(validation_bleu_gr)/len(validation_bleu_gr))

BLEU Score of validation dataset with greedy-search is 0.7565663868271315
